# Brain Networks in Python

BrainNetworksInPython is a tool to perform network analysis over correlation networks of brain regions. 
This tutorial will go through the basic functionality of BrainNetworksInPython, taking us from our inputs (a matrix of structural regional measures over subjects) to a report of local network measures for each brain region, and network level comparisons to a cohort of random graphs of the same degree. 

In [1]:
import numpy as np
import networkx as nx
import BrainNetworksInPython as bnip
import BrainNetworksInPython.datasets as datasets

### Importing data

A BrainNetworksInPython analysis starts with four inputs.
* __regional_measures__
    A pandas DataFrame with subjects as rows. The columns should include structural measures for each brain region, as well as any subject-wise covariates. 
* __names__
    A list of names of the brain regions. This will be used to specify which columns of the __regional_measures__ matrix to want to correlate over.
* __covars__ _(optional)_ 
    A list of your covariates. This will be used to specify which columns of __regional_measure__ you wish to correct for. 
* __centroids__
    A list of tuples representing the cartesian coordinates of brain regions. This list should be in the same order as the list of brain regions to accurately assign coordinates to regions. The coordinates are expected to obey the convention the the x=0 plane is the same plane that separates the left and right hemispheres of the brain. 

In [2]:
# Read in sample data from the NSPN WhitakerVertes PNAS 2016 paper.
df, names, covars, centroids = datasets.NSPN_WhitakerVertes_PNAS2016.import_data()

In [3]:
df.head()

,Unnamed: 0,nspn_id,occ,centre,study_primary,age_scan,sex,male,age_bin,mri_centre,...,rh_supramarginal_part5,rh_supramarginal_part6,rh_supramarginal_part7,rh_frontalpole_part1,rh_temporalpole_part1,rh_transversetemporal_part1,rh_insula_part1,rh_insula_part2,rh_insula_part3,rh_insula_part4
0,0,10356,0,Cambridge,2K_Cohort,20.761,Female,0.0,4,WBIC,...,2.592,2.841,2.318,2.486,3.526,2.638,3.308,2.583,3.188,3.089
1,1,10702,0,Cambridge,2K_Cohort,16.055,Male,1.0,2,WBIC,...,3.448,3.283,2.740,3.225,4.044,3.040,3.867,2.943,3.478,3.609
2,2,10736,0,Cambridge,2K_Cohort,14.897,Female,0.0,1,WBIC,...,3.526,3.269,3.076,3.133,3.900,2.914,3.894,2.898,3.720,3.580
3,3,10778,0,Cambridge,2K_Cohort,20.022,Female,0.0,4,WBIC,...,2.830,2.917,2.647,2.796,3.401,3.045,3.138,2.739,2.833,3.349
4,4,10794,0,Cambridge,2K_Cohort,14.656,Female,0.0,1,WBIC,...,2.689,3.294,2.820,2.539,2.151,2.734,2.791,2.935,3.538,3.403


### Create a correlation matrix
We calculate residuals of the matrix df for the columns of names, correcting for the columns in covars.

In [4]:
df_res = bnip.create_residuals_df(df, names, covars)

Now we create a correlation matrix over the columns of df_res

In [5]:
M = bnip.create_corrmat(df_res, method='pearson')

## Create a weighted graph

A short sidenote on the BrainNetwork class: This is a very lightweight subclass of the [`Networkx.Graph`](https://networkx.github.io/documentation/stable/reference/classes/graph.html) class. This means that any methods you can use on a `Networkx.Graph` object can also be used on a `BrainNetwork` object, although the reverse is not true. We have added various methods which allow us to keep track of measures that have already been calculated, which, especially later on when one is dealing with 10^3 random graphs, saves a lot of time.  
All BrainNetworksInPython measures are implemented in such a way that they can be used on a regular `Networkx.Graph` object. For example, instead of `G.threshold(10)` you can use `bnip.threshold_graph(G, 10)`.  
Also you can create a `BrainNetwork` from a `Networkx.Graph` `G`, using `bnip.BrainNetwork(network=G)`

Initialise a weighted graph `G` from the correlation matrix `M`. The `parcellation` and `centroids` arguments are used to label nodes with names and coordinates respectively. 

In [6]:
G = bnip.BrainNetwork(network=M, parcellation=names, centroids=centroids)

### Threshold to create a binary graph

We threshold G at cost 10 to create a binary graph with 10% as many edges as the complete graph G. Ordinarily when thresholding one takes the 10% of edges with the highest weight. In our case, because we want the resulting graph to be connected, we calculate a minimum spanning tree first. If you want to omit this step, you can pass the argument `mst=False` to `threshold`.
The threshold method does not edit objects inplace

In [7]:
H = G.threshold(10)

### Calculate nodal summary. 

`calculate_nodal_measures` will compute and record the following nodal measures 

* average_dist (if centroids available)
* total_dist (if centroids available)
* betweenness
* closeness
* clustering coefficient
* degree
* interhem (if centroids are available)
* interhem_proportion (if centroids are available)
* nodal partition
* participation coefficient under partition calculated above
* shortest_path_length

`export_nodal_measure` returns nodal attributes in a DataFrame. Let's try it now.

In [8]:
H.export_nodal_measures().head()

,centroids,name,x,y,z
0,"[-27.965157, -19.013702, 17.919528]",lh_bankssts_part1,-27.9652,-19.0137,17.9195
1,"[-14.455663, -13.693461, 13.713674]",lh_bankssts_part2,-14.4557,-13.6935,13.7137
2,"[-33.906934, -22.284672, -15.821168]",lh_caudalanteriorcingulate_part1,-33.9069,-22.2847,-15.8212
3,"[-17.305373, -53.431573, -36.017154]",lh_caudalmiddlefrontal_part1,-17.3054,-53.4316,-36.0172
4,"[-22.265823, -64.366296, -37.674831]",lh_caudalmiddlefrontal_part2,-22.2658,-64.3663,-37.6748


Use `calculate_nodal_measures` to fill in a bunch of nodal measures

In [9]:
H.calculate_nodal_measures()

        Calculating participation coefficient -           may take a little while


In [10]:
H.export_nodal_measures().head()

,average_dist,betweenness,centroids,closeness,clustering,degree,interhem,interhem_proportion,module,name,participation_coefficient,shortest_path_length,total_dist,x,y,z
0,54.5883,0.00824713,"[-27.965157, -19.013702, 17.919528]",0.495961,0.3358,47,18,0.382979,0,lh_bankssts_part1,0.717067,0.00824713,2565.65,-27.9652,-19.0137,17.9195
1,51.7358,0.0124798,"[-14.455663, -13.693461, 13.713674]",0.507438,0.278788,55,17,0.309091,0,lh_bankssts_part2,0.809587,0.0124798,2845.47,-14.4557,-13.6935,13.7137
2,56.9721,0,"[-33.906934, -22.284672, -15.821168]",0.336254,1,2,0,0,1,lh_caudalanteriorcingulate_part1,0.75,0,113.944,-33.9069,-22.2847,-15.8212
3,83.3625,0.0120765,"[-17.305373, -53.431573, -36.017154]",0.525685,0.383485,83,38,0.457831,2,lh_caudalmiddlefrontal_part1,0.459864,0.0120765,6919.09,-17.3054,-53.4316,-36.0172
4,86.0597,0.0292617,"[-22.265823, -64.366296, -37.674831]",0.549195,0.293617,95,39,0.410526,2,lh_caudalmiddlefrontal_part2,0.688753,0.0292617,8175.67,-22.2658,-64.3663,-37.6748


We can also add measures as one might normally add nodal attributes to a networkx graph

In [11]:
nx.set_node_attributes(H, name="hat", values={x: x**2 for x in H.nodes})

These show up in our DataFrame too

In [12]:
H.export_nodal_measures(columns=['name', 'degree', 'hat']).head()

,degree,hat,name
0,47,0,lh_bankssts_part1
1,55,1,lh_bankssts_part2
2,2,4,lh_caudalanteriorcingulate_part1
3,83,9,lh_caudalmiddlefrontal_part1
4,95,16,lh_caudalmiddlefrontal_part2


### Calculate Global measures

In [13]:
H.calculate_global_measures()

{'assortativity': 0.09076922258276784,
 'average_clustering': 0.4498887255891581,
 'average_shortest_path_length': 2.376242649858285,
 'efficiency': 0.47983958611582617,
 'modularity': 0.3828553111606414}

In [14]:
H.calculate_rich_club();

## Create a GraphBundle

The `GraphBundle` object is the BrainNetworksInPython way to handle across network comparisons. What is it? Essentially it's a python dictionary with `BrainNetwork` objects as values. 

In [15]:
brain_bundle = bnip.GraphBundle([H], ['NSPN_cost=10'])

This creates a dictionary-like object with BrainNetwork `H` keyed by `'NSPN_cost=10'`

In [16]:
brain_bundle

{'NSPN_cost=10': <BrainNetworksInPython.classes.BrainNetwork at 0x7f7f78e9ada0>}

Now add a series of random_graphs created by edge swap randomisation of H (keyed by `'NSPN_cost=10'`)

In [17]:
# Note that 10 is not usually a sufficient number of random graphs to do meaningful analysis,
# it is used here for time considerations
brain_bundle.create_random_graphs('NSPN_cost=10', 10)

        Creating 10 random graphs - may take a little while


In [18]:
brain_bundle

{'NSPN_cost=10': <BrainNetworksInPython.classes.BrainNetwork at 0x7f7f78e9ada0>,
 1: <BrainNetworksInPython.classes.BrainNetwork at 0x7f7f786011d0>,
 2: <BrainNetworksInPython.classes.BrainNetwork at 0x7f7f78601208>,
 3: <BrainNetworksInPython.classes.BrainNetwork at 0x7f7f7107b2e8>,
 4: <BrainNetworksInPython.classes.BrainNetwork at 0x7f7f7107b1d0>,
 5: <BrainNetworksInPython.classes.BrainNetwork at 0x7f7f7107b400>,
 6: <BrainNetworksInPython.classes.BrainNetwork at 0x7f7f7107b828>,
 7: <BrainNetworksInPython.classes.BrainNetwork at 0x7f7f7107b160>,
 8: <BrainNetworksInPython.classes.BrainNetwork at 0x7f7f7107b5c0>,
 9: <BrainNetworksInPython.classes.BrainNetwork at 0x7f7f7107b588>,
 10: <BrainNetworksInPython.classes.BrainNetwork at 0x7f7f7107b2b0>}

### Report on a GraphBundle

The following method will calculate global measures ( if they have not already been calculated) for all of the graphs in `graph_bundle` and report the results in a DataFrame. We can do the same for rich club coefficients below.

In [19]:
brain_bundle.report_global_measures()

,assortativity,average_clustering,average_shortest_path_length,efficiency,modularity
NSPN_cost=10,0.090769,0.449889,2.376243,0.479840,0.382855
1,-0.071249,0.235727,2.093193,0.518252,0.000000
2,-0.098295,0.226066,2.080397,0.520099,0.000000
3,-0.077308,0.213056,2.088879,0.518763,0.000000
4,-0.088068,0.238785,2.086594,0.519347,0.000000
5,-0.070630,0.227094,2.092855,0.518107,0.000000
6,-0.081738,0.231484,2.091057,0.518607,0.000000
7,-0.074692,0.230248,2.094801,0.517963,0.000000
8,-0.083732,0.227024,2.090317,0.518569,0.000000
9,-0.092003,0.234767,2.083273,0.519629,0.000000


In [20]:
brain_bundle.report_rich_club()

,0,1,2,3,4,5,6,7,8,9,...,96,97,98,99,100,101,102,103,104,105
NSPN_cost=10,0.100004,0.103228,0.107244,0.112039,0.117842,0.122398,0.127975,0.131899,0.136820,0.141069,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
1,0.100004,0.103228,0.107175,0.111920,0.117564,0.121950,0.127226,0.131092,0.135825,0.139908,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
2,0.100004,0.103228,0.107175,0.111920,0.117564,0.121950,0.127226,0.131092,0.135885,0.139940,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
3,0.100004,0.103228,0.107175,0.111920,0.117564,0.121950,0.127226,0.131150,0.135885,0.139940,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.100004,0.103228,0.107175,0.111920,0.117589,0.121976,0.127254,0.131121,0.135915,0.139940,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
5,0.100004,0.103228,0.107175,0.111920,0.117564,0.121950,0.127226,0.131092,0.135855,0.139908,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,0.100004,0.103228,0.107175,0.111920,0.117564,0.121950,0.127254,0.131121,0.135855,0.139971,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
7,0.100004,0.103228,0.107175,0.111920,0.117564,0.121976,0.127282,0.131208,0.135976,0.140065,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
8,0.100004,0.103228,0.107175,0.111920,0.117564,0.121950,0.127282,0.131150,0.135915,0.140034,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
9,0.100004,0.103228,0.107175,0.111920,0.117564,0.121950,0.127226,0.131092,0.135825,0.139877,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
